In [1]:
# -*- coding:utf-8 -*-
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
from sklearn.model_selection import GridSearchCV  
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [2]:
trainpath = os.getcwd().split('code')[0]
user_info = pd.read_table(trainpath+'train/user_info_train.txt',names=['id','sex','job','education','marriage','hukou'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)
overdue = pd.read_table(trainpath+'train/overdue_train.txt',names=['id','overdue'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)
train = pd.merge(overdue,user_info)

In [3]:
user_info_test = pd.read_table(trainpath+'test/user_info_test.txt',names=['id','sex','job','education','marriage','hukou'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)
usersID_test = pd.read_table(trainpath+'test/usersID_test.txt',names=['id'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)

test = pd.merge(usersID_test,user_info_test)

In [4]:
test.head()

,id,sex,job,education,marriage,hukou
0,55597,2,2,3,1,4
1,55598,1,2,2,3,4
2,55599,2,2,4,2,1
3,55600,2,4,4,3,1
4,55601,1,2,3,1,4


## 放贷款时间loan_time.txt

In [5]:
loan_time_train = pd.read_table(trainpath+'train/loan_time_train.txt',names=['id','loan_time_stamp'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)
train = pd.merge(train,loan_time_train)
train['loan_time'] = train['loan_time_stamp'].apply(lambda x: time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(float(x))))
del train['loan_time_stamp']

In [6]:
loan_time_test = pd.read_table(trainpath+'test/loan_time_test.txt',names=['id','loan_time_stamp'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)
test = pd.merge(test,loan_time_test)
test['loan_time'] = test['loan_time_stamp'].apply(lambda x: time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(float(x))))
del test['loan_time_stamp']

In [7]:
def loan_time(train,test):
    for ds in [train,test]:
        ds['loan_month'] = ds['loan_time'].apply(lambda x: str(x).split('-')[1])
        ds['loan_day'] = ds['loan_time'].apply(lambda x: str(x).split('-')[2].split(' ')[0])
        del ds['loan_time']
    return train,test

In [8]:
train, test = loan_time(train, test)

## 信用卡账单 bill_detail.txt

In [9]:
bill_detail_train = pd.read_table(trainpath+'train/bill_detail_train.txt',names=['id','bill_time_stamp','bank_id','last_bill_money','last_repay_money','card_limit','current_bill_balance','current_lowest_repay_money','pay_count','current_bill_money','adjust_money','recurrent_interest','available_balance','borrow_cash_limit','repay_condition'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)
bill_detail_test = pd.read_table(trainpath+'test/bill_detail_test.txt',names=['id','bill_time_stamp','bank_id','last_bill_money','last_repay_money','card_limit','current_bill_balance','current_lowest_repay_money','pay_count','current_bill_money','adjust_money','recurrent_interest','available_balance','borrow_cash_limit','repay_condition'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)

### train['bill_count'] 

In [10]:
bill_detal_id_value_count_train = bill_detail_train['id'].value_counts()
bill_detal_id_value_count_test = bill_detail_test['id'].value_counts()
train['bill_count'] = train['id'].apply(lambda x: bill_detal_id_value_count_train.ix[x] if x  in bill_detal_id_value_count_train.index  else 0)
test['bill_count'] = test['id'].apply(lambda x: bill_detal_id_value_count_test.ix[x] if x  in bill_detal_id_value_count_test.index  else 0)

### train['if_have_bill_detail']

In [11]:
bill_detail_train_id_unique = bill_detail_train['id'].unique()
train['if_have_bill_detail'] = train['id'].apply(lambda x: 1 if x in bill_detail_train_id_unique  else 0 )
no_bill_detail_train_id_unique = train[train['if_have_bill_detail']==0]['id'].unique()

bill_detail_test_id_unique = bill_detail_test['id'].unique()
test['if_have_bill_detail'] = test['id'].apply(lambda x: 1 if x in bill_detail_test_id_unique  else 0 )
no_bill_detail_test_id_unique = test[test['if_have_bill_detail']==0]['id'].unique()

### train['bill_year'] train['bill_day'] train['bill_month']

In [12]:
bill_detail_train['bill_time'] = bill_detail_train['bill_time_stamp'].apply(lambda x: time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(float(x))))
bill_detail_test['bill_time'] = bill_detail_test['bill_time_stamp'].apply(lambda x: time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(float(x))))

#bill_detail_train['bill_time'].replace('1970-01-01 08:00:00','0000-00-00 00:00:00',inplace = True)
#bill_detail_test['bill_time'].replace('1970-01-01 08:00:00','0000-00-00 00:00:00',inplace = True)


In [13]:
bill_detail_train['bill_year'] = bill_detail_train['bill_time'].apply(lambda x: str(x).split('-')[0])
bill_detail_train['bill_month'] = bill_detail_train['bill_time'].apply(lambda x: str(x).split('-')[1])
bill_detail_train['bill_day'] = bill_detail_train['bill_time'].apply(lambda x: str(x).split('-')[2].split(' ')[0])
bill_detail_train = pd.concat((bill_detail_train,pd.get_dummies(bill_detail_train['bill_year'],prefix='bill_year_count')),axis=1)
bill_detail_train = pd.concat((bill_detail_train,pd.get_dummies(bill_detail_train['bill_month'],prefix='bill_month_count')),axis=1)

bill_detail_test['bill_year'] = bill_detail_test['bill_time'].apply(lambda x: str(x).split('-')[0])
bill_detail_test['bill_month'] = bill_detail_test['bill_time'].apply(lambda x: str(x).split('-')[1])
bill_detail_test['bill_day'] = bill_detail_test['bill_time'].apply(lambda x: str(x).split('-')[2].split(' ')[0])
bill_detail_test = pd.concat((bill_detail_test,pd.get_dummies(bill_detail_test['bill_year'],prefix='bill_year_count')),axis=1)
bill_detail_test = pd.concat((bill_detail_test,pd.get_dummies(bill_detail_test['bill_month'],prefix='bill_month_count')),axis=1)

In [14]:
def get_bill_year_number(x):
    if x % 1000 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bill_year_number].sum()

if os.path.exists('bill_year_number_train.csv'):
    print 'bill_year_number_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('bill_year_number_train.csv'))
else:    
    for bill_year_number in pd.get_dummies(bill_detail_train['bill_year'],prefix='bill_year_count').columns.unique():
        print bill_year_number
        train[bill_year_number] = train['id'].apply(lambda x: get_bill_year_number(x) )
        
def get_bill_year_number_test(x):
    if x % 1000 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bill_year_number].sum()

if os.path.exists('bill_year_number_test.csv'):
    print 'bill_year_number_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('bill_year_number_test.csv'))
else:    
    for bill_year_number in pd.get_dummies(bill_detail_test['bill_year'],prefix='bill_year_count').columns.unique():
        print bill_year_number
        test[bill_year_number] = test['id'].apply(lambda x: get_bill_year_number_test(x) )

bill_year_number_train.csv 存在 
bill_year_number_test.csv 存在 


In [15]:
def save_bill_year_number():
    csv_save_bill_year_number = pd.concat((train['id'],train.loc[:,pd.get_dummies(bill_detail_train['bill_year'],prefix='bill_year_count').columns.unique()]),axis=1)
    csv_save_bill_year_number.to_csv('bill_year_number_train.csv', sep=",", index = False)
#save_bill_year_number()

def save_bill_year_number_test():
    csv_save_bill_year_number = pd.concat((test['id'],test.loc[:,pd.get_dummies(bill_detail_test['bill_year'],prefix='bill_year_count').columns.unique()]),axis=1)
    csv_save_bill_year_number.to_csv('bill_year_number_test.csv', sep=",", index = False)
#save_bill_year_number_test()

In [16]:
def get_bill_month_number(x):
    if x % 1000 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bill_month_number].sum()

if os.path.exists('bill_month_number_train.csv'):
    print 'bill_month_number_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('bill_month_number_train.csv'))
else:    
    for bill_month_number in pd.get_dummies(bill_detail_train['bill_month'],prefix='bill_month_count').columns.unique():
        print bill_month_number
        train[bill_month_number] = train['id'].apply(lambda x: get_bill_month_number(x) )

def get_bill_month_number_test(x):
    if x % 1000 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bill_month_number].sum()

if os.path.exists('bill_month_number_test.csv'):
    print 'bill_month_number_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('bill_month_number_test.csv'))
else:    
    for bill_month_number in pd.get_dummies(bill_detail_test['bill_month'],prefix='bill_month_count').columns.unique():
        print bill_month_number
        test[bill_month_number] = test['id'].apply(lambda x: get_bill_month_number_test(x) )

bill_month_number_train.csv 存在 
bill_month_number_test.csv 存在 


In [17]:
def save_bill_month_number():
    csv_save_bill_month_number = pd.concat((train['id'],train.loc[:,pd.get_dummies(bill_detail_train['bill_month'],prefix='bill_month_count').columns.unique()]),axis=1)
    csv_save_bill_month_number.to_csv('bill_month_number_train.csv', sep=",", index = False)
#save_bill_month_number()

def save_bill_month_number_test():
    csv_save_bill_month_number = pd.concat((test['id'],test.loc[:,pd.get_dummies(bill_detail_test['bill_month'],prefix='bill_month_count').columns.unique()]),axis=1)
    csv_save_bill_month_number.to_csv('bill_month_number_test.csv', sep=",", index = False)
#save_bill_month_number_test()

### train['bank_id_number_**']

In [18]:
bill_detail_train['bank_id'].ix[~bill_detail_train['bank_id'].isin([7,14,4,16,6,3,10,2,8,11,13,15,9,5,1,12])] = 51
bill_detail_test['bank_id'].ix[~bill_detail_test['bank_id'].isin([7,14,4,16,6,3,10,2,8,11,13,15,9,5,1,12])] = 51
bill_detail_train = pd.concat((bill_detail_train,pd.get_dummies(bill_detail_train['bank_id'],prefix='bank_id_number')),axis=1)
bill_detail_test = pd.concat((bill_detail_test,pd.get_dummies(bill_detail_test['bank_id'],prefix='bank_id_number')),axis=1)


/Users/wakemeup/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
if os.path.exists('bank_id_number_train.csv'):
    print 'bank_id_number_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('bank_id_number_train.csv'))
else:
    for bank_id_number in pd.get_dummies(bill_detail_train['bank_id'],prefix='bank_id_number').columns.unique():
        print bank_id_number
        train[bank_id_number] = train['id'].apply(lambda x: bill_detail_train[bill_detail_train['id'] == x][bank_id_number].sum())


bank_id_number_train.csv 存在 


In [20]:
def bank_id_number_train(x):
    if x % 1000 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bank_id_number].sum()

if os.path.exists('bank_id_number_train.csv'):
    print 'bank_id_number_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('bank_id_number_train.csv'))
else:
    for bank_id_number in pd.get_dummies(bill_detail_train['bank_id'],prefix='bank_id_number').columns.unique():
        print bank_id_number
        train[bank_id_number] = train['id'].apply(lambda x: bank_id_number_train(x))

def bank_id_number_test(x):
    if x % 1000 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bank_id_number].sum()

if os.path.exists('bank_id_number_test.csv'):
    print 'bank_id_number_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('bank_id_number_test.csv'))
else:
    for bank_id_number in pd.get_dummies(bill_detail_test['bank_id'],prefix='bank_id_number').columns.unique():
        print bank_id_number
        test[bank_id_number] = test['id'].apply(lambda x: bank_id_number_test(x))

bank_id_number_train.csv 存在 
bank_id_number_test.csv 存在 


In [21]:
def save_bank_id_number_train():
    csv_save_bank_id_number_train = pd.concat((train['id'],train.loc[:,pd.get_dummies(bill_detail_train['bank_id'],prefix='bank_id_number').columns.unique()]),axis=1)
    csv_save_bank_id_number_train.to_csv('bank_id_number_train.csv', sep=",", index = False)
#save_bank_id_number_train()

def save_bank_id_number_test():
    csv_save_bank_id_number_test = pd.concat((test['id'],test.loc[:,pd.get_dummies(bill_detail_test['bank_id'],prefix='bank_id_number').columns.unique()]),axis=1)
    csv_save_bank_id_number_test.to_csv('bank_id_number_test.csv', sep=",", index = False)
#save_bank_id_number_test()

### train['diff_bank_count']

In [22]:
bank_id_number_column_unique = pd.get_dummies(bill_detail_train['bank_id'],prefix='bank_id_number').columns.unique()
def get_diff_bank_count_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bank_id_number_column_unique].max().sum()

if os.path.exists('diff_bank_count_train.csv'):
    print 'diff_bank_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('diff_bank_count_train.csv'))
else:
    train['diff_bank_count'] = train['id'].apply(lambda x: get_diff_bank_count_train(x))


bank_id_number_column_unique = pd.get_dummies(bill_detail_test['bank_id'],prefix='bank_id_number').columns.unique()
def get_diff_bank_count_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bank_id_number_column_unique].max().sum()

if os.path.exists('diff_bank_count_test.csv'):
    print 'diff_bank_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('diff_bank_count_test.csv'))
else:
    test['diff_bank_count'] = test['id'].apply(lambda x: get_diff_bank_count_test(x))


diff_bank_count_train.csv 存在 
diff_bank_count_test.csv 存在 


In [23]:
#bank_id_number_column_unique = pd.get_dummies(bill_detail_train['bank_id'],prefix='bank_id_number').columns.unique()
#bill_detail_train[bank_id_number_column_unique].head()

In [24]:
def save_diff_bank_count_train():
    csv_save_diff_bank_count_train = pd.concat((train['id'],train.loc[:,'diff_bank_count']),axis=1)
    csv_save_diff_bank_count_train.to_csv('diff_bank_count_train.csv', sep=",", index = False)
#save_diff_bank_count_train()

def save_diff_bank_count_test():
    csv_save_diff_bank_count_test = pd.concat((test['id'],test.loc[:,'diff_bank_count']),axis=1)
    csv_save_diff_bank_count_test.to_csv('diff_bank_count_test.csv', sep=",", index = False)
#save_diff_bank_count_test()

### train['last_bill_money_avg']

In [25]:
def get_last_bill_money_avg_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x]['last_bill_money'].mean()

if os.path.exists('last_bill_money_avg_train.csv'):
    print 'last_bill_money_avg_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('last_bill_money_avg_train.csv'))
else:
    train['last_bill_money_avg'] = train['id'].apply(lambda x: get_last_bill_money_avg_train(x))
    
def get_last_bill_money_avg_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x]['last_bill_money'].mean()

if os.path.exists('last_bill_money_avg_test.csv'):
    print 'last_bill_money_avg_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('last_bill_money_avg_test.csv'))
else:
    test['last_bill_money_avg'] = test['id'].apply(lambda x: get_last_bill_money_avg_test(x))

last_bill_money_avg_train.csv 存在 
last_bill_money_avg_test.csv 存在 


In [26]:
#test.info()

In [27]:
def save_last_bill_money_avg_train():
    csv_save_last_bill_money_avg_train = pd.concat((train['id'],train.loc[:,'last_bill_money_avg']),axis=1)
    csv_save_last_bill_money_avg_train.to_csv('last_bill_money_avg_train.csv', sep=",", index = False)
#save_last_bill_money_avg_train()

def save_last_bill_money_avg_test():
    csv_save_last_bill_money_avg_test = pd.concat((test['id'],test.loc[:,'last_bill_money_avg']),axis=1)
    csv_save_last_bill_money_avg_test.to_csv('last_bill_money_avg_test.csv', sep=",", index = False)
#save_last_bill_money_avg_test()

In [28]:
train = pd.concat((train,pd.get_dummies(pd.qcut(train['last_bill_money_avg'],15),prefix='last_bill_money_avg')),axis=1)
test = pd.concat((test,pd.get_dummies(pd.qcut(train['last_bill_money_avg'],15)[:test.shape[0]],prefix='last_bill_money_avg')),axis=1)
del train['last_bill_money_avg']
del test['last_bill_money_avg']

### train['last_rest_repay_money_avg']

In [29]:

def get_last_rest_repay_money_avg_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x]['last_rest_repay_money'].mean()
if os.path.exists('last_rest_repay_money_avg_train.csv'):
    print 'last_rest_repay_money_avg_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('last_rest_repay_money_avg_train.csv'))
else:
    bill_detail_train['last_rest_repay_money'] =bill_detail_train['last_bill_money'] - bill_detail_train['last_repay_money'] 
    train['last_rest_repay_money_avg'] = train['id'].apply(lambda x: get_last_rest_repay_money_avg_train(x))
    
def get_last_rest_repay_money_avg_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x]['last_rest_repay_money'].mean()
if os.path.exists('last_rest_repay_money_avg_test.csv'):
    print 'last_rest_repay_money_avg_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('last_rest_repay_money_avg_test.csv'))
else:
    bill_detail_test['last_rest_repay_money'] =bill_detail_test['last_bill_money'] - bill_detail_test['last_repay_money'] 
    test['last_rest_repay_money_avg'] = test['id'].apply(lambda x: get_last_rest_repay_money_avg_test(x))

last_rest_repay_money_avg_train.csv 存在 
last_rest_repay_money_avg_test.csv 存在 


In [30]:
def save_last_rest_repay_money_avg_train():
    csv_save_last_bill_money_avg_train = pd.concat((train['id'],train.loc[:,'last_rest_repay_money_avg']),axis=1)
    csv_save_last_bill_money_avg_train.to_csv('last_rest_repay_money_avg_train.csv', sep=",", index = False)
#save_last_rest_repay_money_avg_train()

def save_last_rest_repay_money_avg_test():
    csv_save_last_bill_money_avg_test = pd.concat((test['id'],test.loc[:,'last_rest_repay_money_avg']),axis=1)
    csv_save_last_bill_money_avg_test.to_csv('last_rest_repay_money_avg_test.csv', sep=",", index = False)
#save_last_rest_repay_money_avg_test()

In [31]:
train = pd.concat((train,pd.get_dummies(pd.qcut(train['last_rest_repay_money_avg'],14),prefix='last_rest_repay_money_avg')),axis=1)
test = pd.concat((test,pd.get_dummies(pd.qcut(train['last_rest_repay_money_avg'],14)[:test.shape[0]],prefix='last_rest_repay_money_avg')),axis=1)
del train['last_rest_repay_money_avg']
del test['last_rest_repay_money_avg']

### train['card_limit_avg']

In [32]:
def get_card_limit_avg_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x]['card_limit'].mean()
if os.path.exists('card_limit_avg_train.csv'):
    print 'card_limit_avg_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('card_limit_avg_train.csv'))
else:
    train['card_limit_avg'] = train['id'].apply(lambda x: get_card_limit_avg_train(x))
 
def get_card_limit_avg_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x]['card_limit'].mean()
if os.path.exists('card_limit_avg_test.csv'):
    print 'card_limit_avg_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('card_limit_avg_test.csv'))
else:
    test['card_limit_avg'] = test['id'].apply(lambda x: get_card_limit_avg_test(x))
 




card_limit_avg_train.csv 存在 
55600
55700
55800
55900
56000
56100
56200
56300
56400
56500
56600
56700
56800
56900
57000
57100
57200
57300
57400
57500
57600
57700
57800
57900
58000
58100
58200
58300
58400
58500
58600
58700
58800
58900
59000
59100
59200
59300
59400
59500
59600
59700
59800
59900
60000
60100
60200
60300
60400
60500
60600
60700
60800
60900
61000
61100
61200
61300
61400
61500
61600
61700
61800
61900
62000
62100
62200
62300
62400
62500
62600
62700
62800
62900
63000
63100
63200
63300
63400
63500
63600
63700
63800
63900
64000
64100
64200
64300
64400
64500
64600
64700
64800
64900
65000
65100
65200
65300
65400
65500
65600
65700
65800
65900
66000
66100
66200
66300
66400
66500
66600
66700
66800
66900
67000
67100
67200
67300
67400
67500
67600
67700
67800
67900
68000
68100
68200
68300
68400
68500
68600
68700
68800
68900
69000
69100
69200
69300
69400


In [33]:
def save_card_limit_avg_train():
    csv_save_last_bill_money_avg_train = pd.concat((train['id'],train.loc[:,'card_limit_avg']),axis=1)
    csv_save_last_bill_money_avg_train.to_csv('card_limit_avg_train.csv', sep=",", index = False)
save_card_limit_avg_train()

def save_card_limit_avg_test():
    csv_save_last_bill_money_avg_test = pd.concat((test['id'],test.loc[:,'card_limit_avg']),axis=1)
    csv_save_last_bill_money_avg_test.to_csv('card_limit_avg_test.csv', sep=",", index = False)
save_card_limit_avg_train()

In [34]:
train = pd.concat((train,pd.get_dummies(pd.qcut(train['card_limit_avg'],12),prefix='card_limit_avg')),axis=1)
test = pd.concat((test,pd.get_dummies(pd.qcut(train['card_limit_avg'],12)[:test.shape[0]],prefix='card_limit_avg')),axis=1)
del train['card_limit_avg']
del test['card_limit_avg']


#print train['overdue'].groupby(pd.qcut(train['card_limit_avg'],12)).mean()

### train['last_rest_repay_money_morethan0_count'] \ train['last_rest_repay_money_morethan10_count']

In [35]:

def get_last_rest_repay_money_morethan0_count_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bill_detail_train['last_rest_repay_money']>0].shape[0]
if os.path.exists('last_rest_repay_money_morethan0_count_train.csv'):
    print 'last_rest_repay_money_morethan0_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('last_rest_repay_money_morethan0_count_train.csv'))
else:
    bill_detail_train['last_rest_repay_money'] =bill_detail_train['last_bill_money'] - bill_detail_train['last_repay_money'] 
    train['last_rest_repay_money_morethan0_count'] = train['id'].apply(lambda x: get_last_rest_repay_money_morethan0_count_train(x))
    
def get_last_rest_repay_money_morethan0_count_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bill_detail_test['last_rest_repay_money']>0].shape[0]
if os.path.exists('last_rest_repay_money_morethan0_count_test.csv'):
    print 'last_rest_repay_money_morethan0_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('last_rest_repay_money_morethan0_count_test.csv'))
else:
    bill_detail_test['last_rest_repay_money'] =bill_detail_test['last_bill_money'] - bill_detail_test['last_repay_money'] 
    test['last_rest_repay_money_morethan0_count'] = test['id'].apply(lambda x: get_last_rest_repay_money_morethan0_count_test(x))
 

last_rest_repay_money_morethan0_count_train.csv 存在 
last_rest_repay_money_morethan0_count_test.csv 存在 


In [36]:
def save_last_rest_repay_money_morethan0_count_train():
    csv_save_last_rest_repay_money_morethan0_count_train = pd.concat((train['id'],train.loc[:,'last_rest_repay_money_morethan0_count']),axis=1)
    csv_save_last_rest_repay_money_morethan0_count_train.to_csv('last_rest_repay_money_morethan0_count_train.csv', sep=",", index = False)
#save_last_rest_repay_money_morethan0_count_train()

def save_last_rest_repay_money_morethan0_count_test():
    csv_save_last_rest_repay_money_morethan0_count_test = pd.concat((test['id'],test.loc[:,'last_rest_repay_money_morethan0_count']),axis=1)
    csv_save_last_rest_repay_money_morethan0_count_test.to_csv('last_rest_repay_money_morethan0_count_test.csv', sep=",", index = False)
#save_last_rest_repay_money_morethan0_count_test()

In [37]:

def get_last_rest_repay_money_morethan10_count_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bill_detail_train['last_rest_repay_money']>0].shape[0]
if os.path.exists('last_rest_repay_money_morethan10_count_train.csv'):
    print 'last_rest_repay_money_morethan10_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('last_rest_repay_money_morethan10_count_train.csv'))
else:
    bill_detail_train['last_rest_repay_money'] =bill_detail_train['last_bill_money'] - bill_detail_train['last_repay_money'] 
    train['last_rest_repay_money_morethan10_count'] = train['id'].apply(lambda x: get_last_rest_repay_money_morethan10_count_train(x))
    
def get_last_rest_repay_money_morethan10_count_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bill_detail_test['last_rest_repay_money']>0].shape[0]
if os.path.exists('last_rest_repay_money_morethan10_count_test.csv'):
    print 'last_rest_repay_money_morethan10_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('last_rest_repay_money_morethan10_count_test.csv'))
else:
    bill_detail_test['last_rest_repay_money'] =bill_detail_test['last_bill_money'] - bill_detail_test['last_repay_money'] 
    test['last_rest_repay_money_morethan10_count'] = test['id'].apply(lambda x: get_last_rest_repay_money_morethan10_count_test(x))
 

last_rest_repay_money_morethan10_count_train.csv 存在 
last_rest_repay_money_morethan10_count_test.csv 存在 


In [38]:
def save_last_rest_repay_money_morethan10_count_train():
    csv_save_last_rest_repay_money_morethan10_count_train = pd.concat((train['id'],train.loc[:,'last_rest_repay_money_morethan10_count']),axis=1)
    csv_save_last_rest_repay_money_morethan10_count_train.to_csv('last_rest_repay_money_morethan10_count_train.csv', sep=",", index = False)
#save_last_rest_repay_money_morethan10_count_train()

def save_last_rest_repay_money_morethan10_count_test():
    csv_save_last_rest_repay_money_morethan10_count_test = pd.concat((test['id'],test.loc[:,'last_rest_repay_money_morethan10_count']),axis=1)
    csv_save_last_rest_repay_money_morethan10_count_test.to_csv('last_rest_repay_money_morethan10_count_test.csv', sep=",", index = False)
#save_last_rest_repay_money_morethan10_count_test()

### train['current_bill_balance_avg']

In [39]:
def get_card_limit_avg_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x]['card_limit'].mean()
if os.path.exists('card_limit_avg_train.csv'):
    print 'card_limit_avg_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('card_limit_avg_train.csv'))
else:
    train['card_limit_avg'] = train['id'].apply(lambda x: get_card_limit_avg_train(x))
 
def get_card_limit_avg_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x]['card_limit'].mean()
if os.path.exists('card_limit_avg_test.csv'):
    print 'card_limit_avg_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('card_limit_avg_test.csv'))
else:
    test['card_limit_avg'] = test['id'].apply(lambda x: get_card_limit_avg_test(x))
 

card_limit_avg_train.csv 存在 
55600
55700
55800
55900
56000
56100
56200
56300
56400
56500
56600
56700
56800
56900
57000
57100
57200
57300
57400
57500
57600
57700
57800
57900
58000
58100
58200
58300
58400
58500
58600
58700
58800
58900
59000
59100
59200
59300
59400
59500
59600
59700
59800
59900
60000
60100
60200
60300
60400
60500
60600
60700
60800
60900
61000
61100
61200
61300
61400
61500
61600
61700
61800
61900
62000
62100
62200
62300
62400
62500
62600
62700
62800
62900
63000
63100
63200
63300
63400
63500
63600
63700
63800
63900
64000
64100
64200
64300
64400
64500
64600
64700
64800
64900
65000
65100
65200
65300
65400
65500
65600
65700
65800
65900
66000
66100
66200
66300
66400
66500
66600
66700
66800
66900
67000
67100
67200
67300
67400
67500
67600
67700
67800
67900
68000
68100
68200
68300
68400
68500
68600
68700
68800
68900
69000
69100
69200
69300
69400


In [40]:
def save_card_limit_avg_train():
    csv_save_last_bill_money_avg_train = pd.concat((train['id'],train.loc[:,'card_limit_avg']),axis=1)
    csv_save_last_bill_money_avg_train.to_csv('card_limit_avg_train.csv', sep=",", index = False)
#save_card_limit_avg_train()

def save_card_limit_avg_test():
    csv_save_last_bill_money_avg_test = pd.concat((test['id'],test.loc[:,'card_limit_avg']),axis=1)
    csv_save_last_bill_money_avg_test.to_csv('card_limit_avg_test.csv', sep=",", index = False)
#save_card_limit_avg_train()

In [41]:
train = pd.concat((train,pd.get_dummies(pd.qcut(train['card_limit_avg'],12),prefix='card_limit_avg')),axis=1)
test = pd.concat((test,pd.get_dummies(pd.qcut(train['card_limit_avg'],12)[:test.shape[0]],prefix='card_limit_avg')),axis=1)
del train['card_limit_avg']
del test['card_limit_avg']

### train['current_bill_balance_avg']

In [42]:
def get_current_bill_balance_avg_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x]['card_limit'].mean()
if os.path.exists('current_bill_balance_avg_train.csv'):
    print 'current_bill_balance_avg_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('current_bill_balance_avg_train.csv'))
else:
    train['current_bill_balance_avg'] = train['id'].apply(lambda x: get_current_bill_balance_avg_train(x))
 
def get_current_bill_balance_avg_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x]['card_limit'].mean()
if os.path.exists('current_bill_balance_avg_test.csv'):
    print 'current_bill_balance_avg_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('current_bill_balance_avg_test.csv'))
else:
    test['current_bill_balance_avg'] = test['id'].apply(lambda x: get_current_bill_balance_avg_test(x))
 

current_bill_balance_avg_train.csv 存在 
55600
55700
55800
55900
56000
56100
56200
56300
56400
56500
56600
56700
56800
56900
57000
57100
57200
57300
57400
57500
57600
57700
57800
57900
58000
58100
58200
58300
58400
58500
58600
58700
58800
58900
59000
59100
59200
59300
59400
59500
59600
59700
59800
59900
60000
60100
60200
60300
60400
60500
60600
60700
60800
60900
61000
61100
61200
61300
61400
61500
61600
61700
61800
61900
62000
62100
62200
62300
62400
62500
62600
62700
62800
62900
63000
63100
63200
63300
63400
63500
63600
63700
63800
63900
64000
64100
64200
64300
64400
64500
64600
64700
64800
64900
65000
65100
65200
65300
65400
65500
65600
65700
65800
65900
66000
66100
66200
66300
66400
66500
66600
66700
66800
66900
67000
67100
67200
67300
67400
67500
67600
67700
67800
67900
68000
68100
68200
68300
68400
68500
68600
68700
68800
68900
69000
69100
69200
69300
69400


In [43]:
def save_current_bill_balance_avg_train():
    csv_save_last_bill_money_avg_train = pd.concat((train['id'],train.loc[:,'current_bill_balance_avg']),axis=1)
    csv_save_last_bill_money_avg_train.to_csv('current_bill_balance_avg_train.csv', sep=",", index = False)
save_current_bill_balance_avg_train()

def save_current_bill_balance_avg_test():
    csv_save_last_bill_money_avg_test = pd.concat((test['id'],test.loc[:,'current_bill_balance_avg']),axis=1)
    csv_save_last_bill_money_avg_test.to_csv('current_bill_balance_avg_test.csv', sep=",", index = False)
save_current_bill_balance_avg_train()

In [44]:
train = pd.concat((train,pd.get_dummies(pd.qcut(train['current_bill_balance_avg'],12),prefix='current_bill_balance_avg')),axis=1)
test = pd.concat((test,pd.get_dummies(pd.qcut(train['current_bill_balance_avg'],12)[:test.shape[0]],prefix='current_bill_balance_avg')),axis=1)
del train['current_bill_balance_avg']
del test['current_bill_balance_avg']

### train['current_bill_balance_lessthan0_count']

In [45]:
def get_current_bill_balance_lessthan0_count_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bill_detail_train['current_bill_balance']<0].shape[0]
if os.path.exists('current_bill_balance_lessthan0_count_train.csv'):
    print 'current_bill_balance_lessthan0_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('current_bill_balance_lessthan0_count_train.csv'))
else:
    train['current_bill_balance_lessthan0_count'] = train['id'].apply(lambda x: get_current_bill_balance_lessthan0_count_train(x))

def get_current_bill_balance_lessthan0_count_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bill_detail_test['current_bill_balance']<0].shape[0]
if os.path.exists('current_bill_balance_lessthan0_count_test.csv'):
    print 'current_bill_balance_lessthan0_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('current_bill_balance_lessthan0_count_test.csv'))
else:
    test['current_bill_balance_lessthan0_count'] = test['id'].apply(lambda x: get_current_bill_balance_lessthan0_count_test(x))


current_bill_balance_lessthan0_count_train.csv 存在 
current_bill_balance_lessthan0_count_test.csv 存在 


In [46]:
def save_current_bill_balance_lessthan0_count_train():
    csv_save_current_bill_balance_lessthan0_count_train = pd.concat((train['id'],train.loc[:,'current_bill_balance_lessthan0_count']),axis=1)
    csv_save_current_bill_balance_lessthan0_count_train.to_csv('current_bill_balance_lessthan0_count_train.csv', sep=",", index = False)
#save_current_bill_balance_lessthan0_count_train()

def save_current_bill_balance_lessthan0_count_test():
    csv_save_current_bill_balance_lessthan0_count_test = pd.concat((test['id'],test.loc[:,'current_bill_balance_lessthan0_count']),axis=1)
    csv_save_current_bill_balance_lessthan0_count_test.to_csv('current_bill_balance_lessthan0_count_test.csv', sep=",", index = False)
save_current_bill_balance_lessthan0_count_test()

### train['current_card_limit_avg']

In [47]:
bill_detail_train['current_card_limit'] = bill_detail_train['current_lowest_repay_money']-bill_detail_train['current_bill_money']
bill_detail_test['current_card_limit'] = bill_detail_test['current_lowest_repay_money']-bill_detail_test['current_bill_money']

In [48]:
def get_current_card_limit_avg_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x]['current_card_limit'].mean()
if os.path.exists('current_card_limit_avg_train.csv'):
    print 'current_card_limit_avg_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('current_card_limit_avg_train.csv'))
else:
    train['current_card_limit_avg'] = train['id'].apply(lambda x: get_current_card_limit_avg_train(x))
 
def get_current_card_limit_avg_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x]['current_card_limit'].mean()
if os.path.exists('current_card_limit_avg_test.csv'):
    print 'current_card_limit_avg_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('current_card_limit_avg_test.csv'))
else:
    test['current_card_limit_avg'] = test['id'].apply(lambda x: get_current_card_limit_avg_test(x))
 

current_card_limit_avg_train.csv 存在 
55600
55700
55800
55900
56000
56100
56200
56300
56400
56500
56600
56700
56800
56900
57000
57100
57200
57300
57400
57500
57600
57700
57800
57900
58000
58100
58200
58300
58400
58500
58600
58700
58800
58900
59000
59100
59200
59300
59400
59500
59600
59700
59800
59900
60000
60100
60200
60300
60400
60500
60600
60700
60800
60900
61000
61100
61200
61300
61400
61500
61600
61700
61800
61900
62000
62100
62200
62300
62400
62500
62600
62700
62800
62900
63000
63100
63200
63300
63400
63500
63600
63700
63800
63900
64000
64100
64200
64300
64400
64500
64600
64700
64800
64900
65000
65100
65200
65300
65400
65500
65600
65700
65800
65900
66000
66100
66200
66300
66400
66500
66600
66700
66800
66900
67000
67100
67200
67300
67400
67500
67600
67700
67800
67900
68000
68100
68200
68300
68400
68500
68600
68700
68800
68900
69000
69100
69200
69300
69400


In [49]:
def save_current_card_limit_avg_train():
    csv_save_last_bill_money_avg_train = pd.concat((train['id'],train.loc[:,'current_card_limit_avg']),axis=1)
    csv_save_last_bill_money_avg_train.to_csv('current_card_limit_avg_train.csv', sep=",", index = False)
#save_current_card_limit_avg_train()

def save_current_card_limit_avg_test():
    csv_save_last_bill_money_avg_test = pd.concat((test['id'],test.loc[:,'current_card_limit_avg']),axis=1)
    csv_save_last_bill_money_avg_test.to_csv('current_card_limit_avg_test.csv', sep=",", index = False)
#save_current_card_limit_avg_train()

In [50]:
train = pd.concat((train,pd.get_dummies(pd.qcut(train['current_card_limit_avg'],9),prefix='current_card_limit_avg')),axis=1)
test = pd.concat((test,pd.get_dummies(pd.qcut(train['current_card_limit_avg'],9)[:test.shape[0]],prefix='current_card_limit_avg')),axis=1)
del train['current_card_limit_avg']
del test['current_card_limit_avg']

### train['pay_count']

In [51]:
def get_pay_count_avg_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x]['pay_count'].mean()
if os.path.exists('pay_count_avg_train.csv'):
    print 'pay_count_avg_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('pay_count_avg_train.csv'))
else:
    train['pay_count_avg'] = train['id'].apply(lambda x: get_pay_count_avg_train(x))
 
def get_pay_count_avg_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x]['pay_count'].mean()
if os.path.exists('pay_count_avg_test.csv'):
    print 'pay_count_avg_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('pay_count_avg_test.csv'))
else:
    test['pay_count_avg'] = test['id'].apply(lambda x: get_pay_count_avg_test(x))
 

pay_count_avg_train.csv 存在 
55600
55700
55800
55900
56000
56100
56200
56300
56400
56500
56600
56700
56800
56900
57000
57100
57200
57300
57400
57500
57600
57700
57800
57900
58000
58100
58200
58300
58400
58500
58600
58700
58800
58900
59000
59100
59200
59300
59400
59500
59600
59700
59800
59900
60000
60100
60200
60300
60400
60500
60600
60700
60800
60900
61000
61100
61200
61300
61400
61500
61600
61700
61800
61900
62000
62100
62200
62300
62400
62500
62600
62700
62800
62900
63000
63100
63200
63300
63400
63500
63600
63700
63800
63900
64000
64100
64200
64300
64400
64500
64600
64700
64800
64900
65000
65100
65200
65300
65400
65500
65600
65700
65800
65900
66000
66100
66200
66300
66400
66500
66600
66700
66800
66900
67000
67100
67200
67300
67400
67500
67600
67700
67800
67900
68000
68100
68200
68300
68400
68500
68600
68700
68800
68900
69000
69100
69200
69300
69400


In [52]:
def save_pay_count_avg_train():
    csv_save_last_bill_money_avg_train = pd.concat((train['id'],train.loc[:,'pay_count_avg']),axis=1)
    csv_save_last_bill_money_avg_train.to_csv('pay_count_avg_train.csv', sep=",", index = False)
save_pay_count_avg_train()

def save_pay_count_avg_test():
    csv_save_last_bill_money_avg_test = pd.concat((test['id'],test.loc[:,'pay_count_avg']),axis=1)
    csv_save_last_bill_money_avg_test.to_csv('pay_count_avg_test.csv', sep=",", index = False)
save_pay_count_avg_train()

In [53]:
train = pd.concat((train,pd.get_dummies(pd.cut(train['pay_count_avg'],bins=[-1,0,0.2,1,1.2,1.5,1000]),prefix='pay_count_avg')),axis=1)
test = pd.concat((test,pd.get_dummies(pd.cut(test['pay_count_avg'],bins=[-1,0,0.2,1,1.2,1.5,1000])[:test.shape[0]],prefix='pay_count_avg')),axis=1)
del train['pay_count_avg']
del test['pay_count_avg']

### train['adjust_money_morethan0_count']  train['adjust_money_lessthan0_count']

In [54]:
def get_adjust_money_morethan0_count_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bill_detail_train['adjust_money']>0].shape[0]
if os.path.exists('adjust_money_morethan0_count_train.csv'):
    print 'adjust_money_morethan0_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('adjust_money_morethan0_count_train.csv'))
else:
    train['adjust_money_morethan0_count'] = train['id'].apply(lambda x: get_adjust_money_morethan0_count_train(x))

def get_adjust_money_morethan0_count_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bill_detail_test['adjust_money']>0].shape[0]
if os.path.exists('adjust_money_morethan0_count_test.csv'):
    print 'adjust_money_morethan0_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('adjust_money_morethan0_count_test.csv'))
else:
    test['adjust_money_morethan0_count'] = test['id'].apply(lambda x: get_adjust_money_morethan0_count_test(x))


adjust_money_morethan0_count_train.csv 存在 
adjust_money_morethan0_count_test.csv 存在 


In [55]:
def save_adjust_money_morethan0_count_train():
    csv_save_adjust_money_morethan0_count_train = pd.concat((train['id'],train.loc[:,'adjust_money_morethan0_count']),axis=1)
    csv_save_adjust_money_morethan0_count_train.to_csv('adjust_money_morethan0_count_train.csv', sep=",", index = False)
#save_adjust_money_morethan0_count_train()

def save_adjust_money_morethan0_count_test():
    csv_save_adjust_money_morethan0_count_test = pd.concat((test['id'],test.loc[:,'adjust_money_morethan0_count']),axis=1)
    csv_save_adjust_money_morethan0_count_test.to_csv('adjust_money_morethan0_count_test.csv', sep=",", index = False)
save_adjust_money_morethan0_count_test()

In [56]:
def get_adjust_money_lessthan0_count_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bill_detail_train['adjust_money']>0].shape[0]
if os.path.exists('adjust_money_lessthan0_count_train.csv'):
    print 'adjust_money_lessthan0_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('adjust_money_lessthan0_count_train.csv'))
else:
    train['adjust_money_lessthan0_count'] = train['id'].apply(lambda x: get_adjust_money_lessthan0_count_train(x))

def get_adjust_money_lessthan0_count_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bill_detail_test['adjust_money']>0].shape[0]
if os.path.exists('adjust_money_lessthan0_count_test.csv'):
    print 'adjust_money_lessthan0_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('adjust_money_lessthan0_count_test.csv'))
else:
    test['adjust_money_lessthan0_count'] = test['id'].apply(lambda x: get_adjust_money_lessthan0_count_test(x))


adjust_money_lessthan0_count_train.csv 存在 
adjust_money_lessthan0_count_test.csv 存在 


In [57]:
def save_adjust_money_lessthan0_count_train():
    csv_save_adjust_money_lessthan0_count_train = pd.concat((train['id'],train.loc[:,'adjust_money_lessthan0_count']),axis=1)
    csv_save_adjust_money_lessthan0_count_train.to_csv('adjust_money_lessthan0_count_train.csv', sep=",", index = False)
#save_adjust_money_lessthan0_count_train()

def save_adjust_money_lessthan0_count_test():
    csv_save_adjust_money_lessthan0_count_test = pd.concat((test['id'],test.loc[:,'adjust_money_lessthan0_count']),axis=1)
    csv_save_adjust_money_lessthan0_count_test.to_csv('adjust_money_lessthan0_count_test.csv', sep=",", index = False)
save_adjust_money_lessthan0_count_test()

### train['recurrent_interest_morethan0_count']

In [58]:
def get_recurrent_interest_morethan0_count_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bill_detail_train['recurrent_interest']>0].shape[0]
if os.path.exists('recurrent_interest_morethan0_count_train.csv'):
    print 'recurrent_interest_morethan0_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('recurrent_interest_morethan0_count_train.csv'))
else:
    train['recurrent_interest_morethan0_count'] = train['id'].apply(lambda x: get_recurrent_interest_morethan0_count_train(x))

def get_recurrent_interest_morethan0_count_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bill_detail_test['recurrent_interest']>0].shape[0]
if os.path.exists('recurrent_interest_morethan0_count_test.csv'):
    print 'recurrent_interest_morethan0_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('recurrent_interest_morethan0_count_test.csv'))
else:
    test['recurrent_interest_morethan0_count'] = test['id'].apply(lambda x: get_recurrent_interest_morethan0_count_test(x))


recurrent_interest_morethan0_count_train.csv 存在 
recurrent_interest_morethan0_count_test.csv 存在 


In [59]:
def save_recurrent_interest_morethan0_count_train():
    csv_save_recurrent_interest_morethan0_count_train = pd.concat((train['id'],train.loc[:,'recurrent_interest_morethan0_count']),axis=1)
    csv_save_recurrent_interest_morethan0_count_train.to_csv('recurrent_interest_morethan0_count_train.csv', sep=",", index = False)
save_recurrent_interest_morethan0_count_train()

def save_recurrent_interest_morethan0_count_test():
    csv_save_recurrent_interest_morethan0_count_test = pd.concat((test['id'],test.loc[:,'recurrent_interest_morethan0_count']),axis=1)
    csv_save_recurrent_interest_morethan0_count_test.to_csv('recurrent_interest_morethan0_count_test.csv', sep=",", index = False)
save_recurrent_interest_morethan0_count_test()



### train['available_balance_morethan0_count']

In [60]:
def get_available_balance_morethan0_count_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x][bill_detail_train['available_balance']>0].shape[0]
if os.path.exists('available_balance_morethan0_count_train.csv'):
    print 'available_balance_morethan0_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('available_balance_morethan0_count_train.csv'))
else:
    train['available_balance_morethan0_count'] = train['id'].apply(lambda x: get_available_balance_morethan0_count_train(x))

def get_available_balance_morethan0_count_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x][bill_detail_test['available_balance']>0].shape[0]
if os.path.exists('available_balance_morethan0_count_test.csv'):
    print 'available_balance_morethan0_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('available_balance_morethan0_count_test.csv'))
else:
    test['available_balance_morethan0_count'] = test['id'].apply(lambda x: get_available_balance_morethan0_count_test(x))


available_balance_morethan0_count_train.csv 存在 
available_balance_morethan0_count_test.csv 存在 


In [61]:
def save_available_balance_morethan0_count_train():
    csv_save_available_balance_morethan0_count_train = pd.concat((train['id'],train.loc[:,'available_balance_morethan0_count']),axis=1)
    csv_save_available_balance_morethan0_count_train.to_csv('available_balance_morethan0_count_train.csv', sep=",", index = False)
#save_available_balance_morethan0_count_train()

def save_available_balance_morethan0_count_test():
    csv_save_available_balance_morethan0_count_test = pd.concat((test['id'],test.loc[:,'available_balance_morethan0_count']),axis=1)
    csv_save_available_balance_morethan0_count_test.to_csv('available_balance_morethan0_count_test.csv', sep=",", index = False)
#save_available_balance_morethan0_count_test()



### train['borrow_cash_limit_avg']

In [62]:
def get_borrow_cash_limit_avg_train(x):
    if x % 100 == 0: print x
    return bill_detail_train[bill_detail_train['id'] == x]['borrow_cash_limit'].mean()

if os.path.exists('borrow_cash_limit_avg_train.csv'):
    print 'borrow_cash_limit_avg_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('borrow_cash_limit_avg_train.csv'))
else:
    train['borrow_cash_limit_avg'] = train['id'].apply(lambda x: get_borrow_cash_limit_avg_train(x))
    
def get_borrow_cash_limit_avg_test(x):
    if x % 100 == 0: print x
    return bill_detail_test[bill_detail_test['id'] == x]['borrow_cash_limit'].mean()

if os.path.exists('borrow_cash_limit_avg_test.csv'):
    print 'borrow_cash_limit_avg_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('borrow_cash_limit_avg_test.csv'))
else:
    test['borrow_cash_limit_avg'] = test['id'].apply(lambda x: get_borrow_cash_limit_avg_test(x))

borrow_cash_limit_avg_train.csv 存在 
borrow_cash_limit_avg_test.csv 存在 


In [63]:
def save_borrow_cash_limit_avg_train():
    csv_save_borrow_cash_limit_avg_train = pd.concat((train['id'],train.loc[:,'borrow_cash_limit_avg']),axis=1)
    csv_save_borrow_cash_limit_avg_train.to_csv('borrow_cash_limit_avg_train.csv', sep=",", index = False)
#save_borrow_cash_limit_avg_train()

def save_borrow_cash_limit_avg_test():
    csv_save_borrow_cash_limit_avg_test = pd.concat((test['id'],test.loc[:,'borrow_cash_limit_avg']),axis=1)
    csv_save_borrow_cash_limit_avg_test.to_csv('borrow_cash_limit_avg_test.csv', sep=",", index = False)
#save_borrow_cash_limit_avg_test()

In [64]:
bins2 =  [0,8,17,25]
train = pd.concat((train,pd.get_dummies(pd.cut(train['borrow_cash_limit_avg'],bins=bins2),prefix='borrow_cash_limit_avg')),axis=1)
test = pd.concat((test,pd.get_dummies(pd.cut(test['borrow_cash_limit_avg'],bins=bins2)[:test.shape[0]],prefix='borrow_cash_limit_avg')),axis=1)
del train['borrow_cash_limit_avg']
del test['borrow_cash_limit_avg']

### train['repay_condition_1'] 经过分析很可能没用？？

In [65]:

#test['repay_condition_1'].iloc[60000,:]

In [66]:
train['repay_condition_1'] = 0
repay_condition_1_id = bill_detail_train[bill_detail_train['repay_condition']>0]['id'].unique()
train.loc[train['id'].isin(repay_condition_1_id),'repay_condition_1']=1

test['repay_condition_1'] = 0
repay_condition_1_id = bill_detail_test[bill_detail_test['repay_condition']>0]['id'].unique()
print repay_condition_1_id

test.loc[test['id'].isin(repay_condition_1_id),'repay_condition_1']=1

[64011 60389 65340 57192 56516 60398 63310 68769 59116 68020 68780 62056
 57216 63352 56563 68801 56028 62089 57949 63611 62365 65063 57552 60156
 65870 63762 55776 57703 67158]


## bank_detail.txt

In [67]:
bank_detail_train = pd.read_table(trainpath+'train/bank_detail_train.txt',names=['id','bank_time_stamp','business_type','business_money','wage_sign'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)
bank_detail_test = pd.read_table(trainpath+'test/bank_detail_test.txt',names=['id','bank_time_stamp','business_type','business_money','wage_sign'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)

### train['if_have_bank_detail']

In [68]:
bank_detail_train_id_unique = bank_detail_train['id'].unique()
train['if_have_bank_detail'] = train['id'].apply(lambda x: 1 if x in bank_detail_train_id_unique  else 0 )
no_bank_detail_train_id_unique = train[train['if_have_bank_detail']==0]['id'].unique()

bank_detail_test_id_unique = bank_detail_test['id'].unique()
test['if_have_bank_detail'] = test['id'].apply(lambda x: 1 if x in bank_detail_test_id_unique  else 0 )
no_bank_detail_test_id_unique = test[test['if_have_bank_detail']==0]['id'].unique()

### train['bank_count'] 

In [69]:
bank_detal_id_value_count_train = bank_detail_train['id'].value_counts()
train['bank_count'] = train['id'].apply(lambda x: bank_detal_id_value_count_train.ix[x] if x  in bank_detal_id_value_count_train.index  else 0)

bank_detal_id_value_count_test = bank_detail_test['id'].value_counts()
test['bank_count'] = test['id'].apply(lambda x: bank_detal_id_value_count_test.ix[x] if x  in bank_detal_id_value_count_test.index  else 0)


### train['bank_year_count_**'] 

In [70]:
bank_detail_train['bank_time'] = bank_detail_train['bank_time_stamp'].apply(lambda x: time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(float(x))))
bank_detail_test['bank_time'] = bank_detail_test['bank_time_stamp'].apply(lambda x: time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(float(x))))

In [71]:
bank_detail_train['bank_year'] = bank_detail_train['bank_time'].apply(lambda x: str(x).split('-')[0])
bank_detail_train['bank_month'] = bank_detail_train['bank_time'].apply(lambda x: str(x).split('-')[1])
bank_detail_train['bank_day'] = bank_detail_train['bank_time'].apply(lambda x: str(x).split('-')[2].split(' ')[0])
bank_detail_train['bank_hour'] = bank_detail_train['bank_time'].apply(lambda x: str(x).split(' ')[1].split(':')[0])
bank_detail_train['bank_minutes'] = bank_detail_train['bank_time'].apply(lambda x: str(x).split(':')[1])
#bank_detail_train['bank_day'] = bank_detail_train['bank_time'].apply(lambda x: str(x).split('-')[2].split(' ')[0])

bank_detail_test['bank_year'] = bank_detail_test['bank_time'].apply(lambda x: str(x).split('-')[0])
bank_detail_test['bank_month'] = bank_detail_test['bank_time'].apply(lambda x: str(x).split('-')[1])
bank_detail_test['bank_day'] = bank_detail_test['bank_time'].apply(lambda x: str(x).split('-')[2].split(' ')[0])
bank_detail_test['bank_hour'] = bank_detail_test['bank_time'].apply(lambda x: str(x).split(' ')[1].split(':')[0])
bank_detail_test['bank_minutes'] = bank_detail_test['bank_time'].apply(lambda x: str(x).split(':')[1])
#bank_detail_test['bank_day'] = bank_detail_test['bank_time'].apply(lambda x: str(x).split('-')[2].split(' ')[0])

In [72]:
bank_detail_train = pd.concat((bank_detail_train,pd.get_dummies(bank_detail_train['bank_year'],prefix='bank_year_count')),axis=1)
bank_detail_test = pd.concat((bank_detail_test,pd.get_dummies(bank_detail_test['bank_year'],prefix='bank_year_count')),axis=1)

In [73]:
test.shape[0]

13899

In [74]:
def get_bank_year_number_train(x):
    if x % 100 == 0: print x
    return bank_detail_train[bank_detail_train['id'] == x][bank_year_number].sum()

if os.path.exists('bank_year_number_train.csv'):
    print 'bank_year_number_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('bank_year_number_train.csv'))
else:    
    for bank_year_number in pd.get_dummies(bank_detail_train['bank_year'],prefix='bank_year_count').columns.unique():
        print bank_year_number
        train[bank_year_number] = train['id'].apply(lambda x: get_bank_year_number_train(x) )

def get_bank_year_number_test(x):
    if x % 100 == 0: print x
    return bank_detail_test[bank_detail_test['id'] == x][bank_year_number].sum()

if os.path.exists('bank_year_number_test.csv'):
    print 'bank_year_number_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('bank_year_number_test.csv'))
else:    
    for bank_year_number in pd.get_dummies(bank_detail_test['bank_year'],prefix='bank_year_count').columns.unique():
        print bank_year_number
        test[bank_year_number] = test['id'].apply(lambda x: get_bank_year_number_test(x) )
        print 'over'
        print test[bank_year_number].head()

bank_year_number_train.csv 存在 
bank_year_number_test.csv 存在 


In [75]:
test.shape[0]

13899

In [76]:
def save_bank_year_number():
    csv_save_bank_year_number = pd.concat((train['id'],train.loc[:,pd.get_dummies(bank_detail_train['bank_year'],prefix='bank_year_count').columns.unique()]),axis=1)
    csv_save_bank_year_number.to_csv('bank_year_number_train.csv', sep=",", index = False)
#save_bank_year_number()

def save_bank_year_number_test():
    csv_save_bank_year_number = pd.concat((test['id'],test.loc[:,pd.get_dummies(bank_detail_test['bank_year'],prefix='bank_year_count').columns.unique()]),axis=1)
    csv_save_bank_year_number.to_csv('bank_year_number_test.csv', sep=",", index = False)
#save_bank_year_number_test()

In [77]:
bins3 = [-1,0.5,320,20000]
train = pd.concat((train,pd.get_dummies(pd.cut(train['bank_year_count_2157'],bins=bins3),prefix='bank_year_count_2157')),axis=1)
test = pd.concat((test,pd.get_dummies(pd.cut(test['bank_year_count_2157'],bins=bins3)[:test.shape[0]],prefix='bank_year_count_2157')),axis=1)

In [78]:
del train['bank_year_count_2157']
del test['bank_year_count_2157']

In [79]:
for year in ['bank_year_count_2156','bank_year_count_1970']:
    for ds in [train,test]:
        ds.loc[ds[year]>0,year] = 1


In [80]:
test.shape[0]

13899

### train['business_type_1_count'] train['business_type_0_count']  ..> dummy

In [ ]:
#train['bank_year_count_1970'].head()

In [ ]:
def get_business_type_1_count_train(x):
    if x % 100 == 0: print x
    return bank_detail_train[bank_detail_train['id'] == x][bank_detail_train['business_type']==1].shape[0]
if os.path.exists('business_type_1_count_train.csv'):
    print 'business_type_1_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('business_type_1_count_train.csv'))
else:
    train['business_type_1_count'] = train['id'].apply(lambda x: get_business_type_1_count_train(x))

def get_business_type_1_count_test(x):
    if x % 100 == 0: print x
    return bank_detail_test[bank_detail_test['id'] == x][bank_detail_test['business_type']==1].shape[0]
if os.path.exists('business_type_1_count_test.csv'):
    print 'business_type_1_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('business_type_1_count_test.csv'))
else:
    test['business_type_1_count'] = test['id'].apply(lambda x: get_business_type_1_count_test(x))


/Users/wakemeup/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100


In [ ]:
test.shape[0]

In [ ]:
def save_business_type_1_count_train():
    csv_save_business_type_1_count_train = pd.concat((train['id'],train.loc[:,'business_type_1_count']),axis=1)
    csv_save_business_type_1_count_train.to_csv('business_type_1_count_train.csv', sep=",", index = False)
#save_business_type_1_count_train()

def save_business_type_1_count_test():
    csv_save_business_type_1_count_test = pd.concat((test['id'],test.loc[:,'business_type_1_count']),axis=1)
    csv_save_business_type_1_count_test.to_csv('business_type_1_count_test.csv', sep=",", index = False)
#save_business_type_1_count_test()



In [ ]:
train.loc[train['business_type_1_count']>0,'business_type_1_count'] = 1
test.loc[test['business_type_1_count']>0,'business_type_1_count'] = 1

In [ ]:
test.shape[0]

def get_business_type_0_count_train(x):
    if x % 100 == 0: print x
    return bank_detail_train[bank_detail_train['id'] == x][bank_detail_train['business_type']==1].shape[0]
if os.path.exists('business_type_0_count_train.csv'):
    print 'business_type_0_count_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('business_type_0_count_train.csv'))
else:
    train['business_type_0_count'] = train['id'].apply(lambda x: get_business_type_0_count_train(x))

def get_business_type_0_count_test(x):
    if x % 100 == 0: print x
    return bank_detail_test[bank_detail_test['id'] == x][bank_detail_test['business_type']==1].shape[0]
if os.path.exists('business_type_0_count_test.csv'):
    print 'business_type_0_count_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('business_type_0_count_test.csv'))
else:
    test['business_type_0_count'] = test['id'].apply(lambda x: get_business_type_0_count_test(x))


def save_business_type_0_count_train():
    csv_save_business_type_0_count_train = pd.concat((train['id'],train.loc[:,'business_type_0_count']),axis=1)
    csv_save_business_type_0_count_train.to_csv('business_type_0_count_train.csv', sep=",", index = False)
save_business_type_0_count_train()

def save_business_type_0_count_test():
    csv_save_business_type_0_count_test = pd.concat((test['id'],test.loc[:,'business_type_0_count']),axis=1)
    csv_save_business_type_0_count_test.to_csv('business_type_0_count_test.csv', sep=",", index = False)
save_business_type_0_count_test()



In [ ]:
### train['business_money_avg']

"""
def get_business_money_avg_train(x):
    if x % 100 == 0: print x
    return bank_detail_train[bank_detail_train['id'] == x]['business_money'].mean()

if os.path.exists('business_money_avg_train.csv'):
    print 'business_money_avg_train.csv 存在 '
    train = pd.merge(train,pd.read_csv('business_money_avg_train.csv'))
else:
    train['business_money_avg'] = train['id'].apply(lambda x: get_business_money_avg_train(x))
    
def get_business_money_avg_test(x):
    if x % 100 == 0: print x
    return bank_detail_test[bank_detail_test['id'] == x]['business_money'].mean()

if os.path.exists('business_money_avg_test.csv'):
    print 'business_money_avg_test.csv 存在 '
    test = pd.merge(test,pd.read_csv('business_money_avg_test.csv'))
else:
    test['business_money_avg'] = test['id'].apply(lambda x: get_business_money_avg_test(x))
"""

def save_business_money_avg_train():
    csv_save_business_money_avg_train = pd.concat((train['id'],train.loc[:,'business_money_avg']),axis=1)
    csv_save_business_money_avg_train.to_csv('business_money_avg_train.csv', sep=",", index = False)
save_business_money_avg_train()

def save_business_money_avg_test():
    csv_save_business_money_avg_test = pd.concat((test['id'],test.loc[:,'business_money_avg']),axis=1)
    csv_save_business_money_avg_test.to_csv('business_money_avg_test.csv', sep=",", index = False)
save_business_money_avg_test()

##### test['last_rest_repay_money_avg']

In [ ]:
# 检查test 于train 的column是否匹配
for i in  range(1,len(test.columns.unique())):
    if train.columns.unique()[i+1] != test.columns.unique()[i]:
        print   train.columns.unique()[i+1],test.columns.unique()[i]  

In [ ]:
train.columns.unique()

In [ ]:
#del train['loan_time']
#del test['loan_time']
test.columns.unique()

In [ ]:
test.info()

In [ ]:

rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=1000,
                             min_samples_split=15,
                             min_samples_leaf=2,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                              n_jobs=-1)
rf.fit(train.iloc[:, 2:], train.iloc[:, 1])
#rf.fit(test.iloc[:, 2:], test.iloc[:, 1])
print("%.4f" % rf.oob_score_)

In [ ]:
pd.concat((pd.DataFrame(train.iloc[:, 2:].columns, columns = ['variable']), 
           pd.DataFrame(rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False).to_csv('feature_importance_2.csv', sep=",", index = False)

记录一些历史值：
0.872
0.8722
0.8722
0.8723
0.8723
0.8722 #rf.fit(train.iloc[:, 2:-7], train.iloc[:, 1])
0.8722 #rf.fit(train.iloc[:, 2:], train.iloc[:, 1])
0.8753 加入bank business type
0.8754


In [ ]:
predictions = rf.predict_proba(test.iloc[:,1:])[:,1]
print 'predictions =', predictions
#predictions_overdue_rate = np.sum(predictions)/float(len(predictions))
#print predictions_overdue_rate
predictions = pd.DataFrame(predictions, columns=[u'probability'])
predictions = pd.concat((test['id'], predictions), axis = 1)
predictions.to_csv('y_test_3.csv', sep=",", index = False)

In [ ]:
predictions.shape[0]

In [ ]:
test.shape[0]

In [ ]:
predictions = rf.predict_proba(train.iloc[:,2:])[:,1]
print 'predictions =', predictions
#predictions_overdue_rate = np.sum(predictions)/float(len(predictions))
#print predictions_overdue_rate
predictions = pd.DataFrame(predictions, columns=[u'probability'])
predictions = pd.concat((train['id'], predictions), axis = 1)
predictions.to_csv('y_test_for_ks_score_check.csv', sep=",", index = False)

In [ ]:
len(predictions)

In [ ]:
train.shape[0]